In [92]:
import numpy as np
import sympy as sp
from sympy import Symbol
from sympy import solve, Poly, Eq, Function, exp
import math
import matplotlib.pyplot as plt
from numpy.lib.scimath import sqrt as csqrt
from scipy.signal import find_peaks
import os
import imp
import matplotlib.pyplot as plt
import sys

#windows path
#os.add_dll_directory('C:\\Program Files\\Lumerical\\v202\\api\\python\\') 
#lumapi = imp.load_source('lumapi','C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')

#linux path
#sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
#import lumapi
#sys.path.append(os.path.dirname('/home/moises/Downloads/'))

In [3]:
fdtd = lumapi.FDTD(filename= 'Bragg.fsp')

In [45]:
period = 348.8e-9
N = 150
L = N*period
delta_w = 44.3e-9
w = 450e-9
height = 0.22e-6
material_core = 'Si (Silicon) - Palik Copy 1'
material_subs = 'SiO2 (Glass) - Palik Copy 1'

In [78]:
fdtd.switchtolayout()
fdtd.selectall()
fdtd.delete()

fdtd.addstructuregroup()
fdtd.set('name','corrugation width')

for i in range(N):
    fdtd.addrect()
    fdtd.set({'x min':i*period,
    'x max':i*period + period/2,
    'y':0,
    'y span':w + delta_w,
    'material':material_core})
    fdtd.addtogroup('corrugation width')
    fdtd.addrect()
    fdtd.set({'x min':i*period + period/2,
    'x max':i*period + period/2 + period/2,
    'y':0,
    'y span':w - delta_w,
    'material':material_core})
    fdtd.addtogroup('corrugation width')

fdtd.addrect()
fdtd.set({'name':'rect1',
'x min':-0.5e-6,
'x max':0,
'y':0,
'y span':w,
'z':0,
'z span':height,
'material':material_core})

fdtd.addrect()
fdtd.set({'name':'rect2',
'x min':N*period,
'x max':N*period + 0.5e-6,
'y':0,
'y span':w,
'z':0,
'z span':height,
'material':material_core})

z_span = 4e-6
z_min = -height/2 - z_span
z_max = -height/2
z = (z_min + z_max)/2

fdtd.addrect()
fdtd.set("name", 'substrato')
fdtd.set("x min",-10e-6)
fdtd.set("x max",N*period + 10e-6)
fdtd.set('y',0)
fdtd.set('y span', 10e-6)
fdtd.set('z', z)
fdtd.set('z span', z_span)
fdtd.set('material',material_subs)

In [91]:
fdtd.select('FDTD')
fdtd.delete()
fdtd.addfdtd()
fdtd.set({'x':N*period/2,
'x span':N*period + 0.5e-6,
'y':0,
'y span':3e-6,
'z':0,
'z span':2e-6,
'simulation time':10000e-15,
'x min bc':'PML',
'x max bc':'PML',
'y min bc':'PML',
'y max bc':'PML',
'z min bc':'PML',
'z max bc':'PML'})

fdtd.addport()
fdtd.set({'direction':'Forward',
'x':-0.25e-6,
'y':0,
'y span':2e-6,
'z':0,
'z span':2e-6})
# fdtd.addport()
# fdtd.set({'direction':'Backward',
# 'x':N*period + 0.25e-6,
# 'y':0,
# 'y span':2e-6,
# 'z':0,
# 'z span':2e-6})

fdtd.select('FDTD::ports')
fdtd.set('monitor frequency points', 1000)

fdtd.select('monitor')
fdtd.delete()
fdtd.addpower()
fdtd.set('override global monitor settings',1)
fdtd.set({'frequency points':1000,
'monitor type':'2D X-normal',
'x':-0.2e-6,
'y':0,
'y span':2.5e-6,
'z':0,
'z span':2e-6})